<a href="https://colab.research.google.com/github/AVKuzminov/minor_ida_3_course_2_semestr/blob/master/hw1%20(meteo%20parser).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Работу выполнили студенты ИАД-3:


>  Полякова Ольга, Бурлак Анастасия, Кузьминов Артем



# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]

Выберите произвольным образом город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.

Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание.  

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

**Загрузка библиотек**

В этой части задания мы будем использовать библиотеки beautifulsoup4 и urllub.
Первая позводит нам обработать данные, найти по нужным (заранее опрелеленным на сайте благодаря встроенному инструментарию браузера) тегам информацию для дальнейшей обработки, а urllib - считать ее по ссылке.

In [0]:
#Загрузка библиотек
!pip install beautifulsoup4
import bs4 as bs
import random
import urllib
import pandas as pnd

**Считывание веб-страницы**

In [0]:
#Ссылка на веб-страницу
full_link= "https://www.gismeteo.ru/weather-khabarovsk-4862/10-days/"

In [0]:
#1 Urllib
page = urllib.request.urlopen(full_link)
raw_page = page.read()

**Парсинг**

In [0]:
#2 BeautifulSoup - библиотека для парсинга, поиска по html-тегам
import warnings
warnings.filterwarnings("ignore")
soup = bs.BeautifulSoup(markup=raw_page, exclude_encodings="lxml")

In [0]:
iterator = soup.find_all("div", class_= "widget__container")
days = []
n_days = 1
table = iterator[0] #первый такой элемерт на сайте, в нем вся нужная информация
rows = table.find_all("div", class_ = "widget__row")
#rows[0] - дни
#rows[1] - солнышки
#rows[2] - температуры
#rows[3] - макс. скорость ветра
#rows[4] - осадки

In [0]:
#дни - названия колонок
days_of_week = []
numbers = []
days = rows[0].find_all("div", class_ = "w_date")
numbers_raw = rows[0].find_all(class_ = ["w_date__date black","w_date__date weekend"])
days_of_week_raw = rows[0].find_all("div", class_ = "w_date__day")
for day in days_of_week_raw:
  days_of_week.append(day.contents[0])
for number in numbers_raw:
  numbers.append(number.contents[0].replace(" ","").replace("\n","")[0:2])
columns = []
for i in range(10):
  columns.append(numbers[i] +".02"+ " (" + days_of_week[i].lower() +")")
print(columns)

['18.02 (вс)', '19.02 (пн)', '20.02 (вт)', '21.02 (ср)', '22.02 (чт)', '23.02 (пт)', '24.02 (сб)', '25.02 (вс)', '26.02 (пн)', '27.02 (вт)']


In [0]:
#максимальная и минимальная температуры
max_temperature_raw = rows[2].find_all("div", class_ = "maxt")
min_temperature_raw = rows[2].find_all("div", class_ = "mint")
max_temperature = []
min_temperature = []
for i in range(10):
  max_temperature.append(float(max_temperature_raw[i].contents[0].replace('−','-')))
  min_temperature.append(float(min_temperature_raw[i].contents[0].replace('−','-')))
print(max_temperature)
print(min_temperature)

[-13.0, -9.0, -12.0, -13.0, -12.0, -14.0, -13.0, -13.0, -10.0, -10.0]
[-22.0, -18.0, -19.0, -20.0, -22.0, -22.0, -23.0, -22.0, -20.0, -19.0]


In [0]:
#скорость ветра
wind = []
wind_raw = rows[3].find_all("div", class_ = "w_wind__warning")
for wnd in wind_raw:
  wind.append(float(wnd.contents[1].strip()))
wind

[10.0, 13.0, 12.0, 11.0, 7.0, 6.0, 11.0, 10.0, 6.0, 10.0]

In [0]:
#уровень осадков
rain = []
rain_raw = rows[4].find_all("div", class_ = "w_precipitation__value")
for rn in rain_raw:
  rain.append(float(rn.contents[0].strip().replace(',','.')))
rain

[0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0]

Создание датафрейма

In [0]:
index = ['минимальная температура','максимальная температура', 'скорость ветра', 'уровень осадков']
data = [min_temperature,max_temperature,wind,rain]
df = pnd.DataFrame(columns = columns,data=data, index = index)
df.head()

,18.02 (вс),19.02 (пн),20.02 (вт),21.02 (ср),22.02 (чт),23.02 (пт),24.02 (сб),25.02 (вс),26.02 (пн),27.02 (вт)
минимальная температура,-22.0,-18.0,-19.0,-20.0,-22.0,-22.0,-23.0,-22.0,-20.0,-19.0
максимальная температура,-13.0,-9.0,-12.0,-13.0,-12.0,-14.0,-13.0,-13.0,-10.0,-10.0
скорость ветра,10.0,13.0,12.0,11.0,7.0,6.0,11.0,10.0,6.0,10.0
уровень осадков,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0


### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на ближайшие 10 дней в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

## Выполнение 2 части задания

Во второй части задания мы использовали pymorphy2 - морфологический анализатор, который мы использовали для того, чтобы приводить слова в нужную форму (число, падеж) и правильно согласовывать с числительными. В своем прогнозе мы ипользовали следующие шаблоны:
- Для первого дня:
  - В день1 в НазваниеГорода будет X градус (-а, -ов). 
  - Скорость ветра будет достигать X метр (-а, -ов) в секунду. 
  - Уровень осадков будет равен X миллиметр (-а, -ов).
- Для всех остальных дней:
  - В день(n) в НазваниеГорода [ потеплеет / похолодает на X градус (-а, -ов) ] / [ температура не изменится ] по сравнению с день(n-1). [Температура будет равна X градус (-а, -ов).] 
  - Скорость ветра (не) изменится на X единиц (-, -ы). [ Ветер будет дуть со скоростью X метр (-а, -ов) в секунду.] 
  - Уровень осадков [ повысится / понизится на X единиц (-, -ы) ] / [ не изменится ]. [ Осадков выпадет 0.0 миллиметров. ]
- Итоговые заключения:
  - За N дней / дня в НазваниеГорода [ потеплеет / похолодает на X градус (-а, -ов) ] / [ температура не изменится ]. 
  - Уровень осадков [ повысится / понизится на X единиц (-, -ы) ] / [ не изменится ].
***  
Для получения дневной температуры мы находили среднее между максимальной и минимальной температурой.
***
Также для написания некоторых функций (например, получения названия дня недели и месяца) мы использовали регулярные выражения. Регулярные выражения - формальный язык поиска и осуществления манипуляций с подстроками в тексте, основанный на использовании метасимволов (определение с wikipedia).

***
С помощью пакета pymorphy2 были осуществленны следующие функции:
 - "парсинг" слова: функция morph.parse(word)[0];
 - согласование существительного с числительным: функция make_agree_with_number("число").word из пакета;
 - приведение существительного к нужному падежу: функция inflect({'падеж'}).word из пакета.
 
С помощью библиотеки re - регулярные выражения мы смогли написать функции, реализующие:
 - получение полного названия дня недели из его сокращения - использованы поиск по строке (findall) и удаление символов (sub);
 - получение названия месяца из даты - поиск по строке (findall);
 - согласование предлога с существительным (по правилам русского языка) - поиск по строке (findall).

In [0]:
!pip install pymorphy2

In [0]:
import re
import numpy as np
import pandas as pd
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
# Работа с днями недели (получение полного названия из сокращения)

days = ["понедельник","вторник","среда","четверг","пятница", "суббота", "воскресенье"]

def name_of_day(col):
  day_column = re.sub(r'[.\s\(\)]', '', col)
  short_name = re.findall(r'\D+', day_column)[0]
  for day in days:
    short = re.sub(r'[оеяу]', '', re.findall(r'^\w\w\w', day)[0])
    if short_name == short:
      day_needed = day
  return day_needed

In [0]:
# Получение названия месяца из даты

months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']

def date(col):
  day = int(re.findall(r'\w+', col)[0])
  month = months[int(re.findall(r'\w+', col)[1])-1]
  month = morph.parse(month)[0]
  month = month.inflect({'gent'}).word 
  return str(day)+' '+month

In [0]:
# Согласование предлога с существительным

# если после предлога "в" идет слово, которое также начинается с буквы "в", а после нее идет согласная - пишется предлог "во"
def preposition_v(word):
  find = re.findall(r'^\w\w', word)[0]
  if find[0]== 'в' and find[1]== re.findall(r'[^аеёиоуыэяю]', word)[1]:
    return 'во'
  else:
    return 'в'

# если после предлога "с" идет слова, начинающееся с "с" и согласной после, также перед сочетаниями [в] + согласная - пишется предлог "со"
def preposition_s(word):
  find = re.findall(r'^\w\w', word)[0]
  if (find[0]== 'с' and find[1]== re.findall(r'[^аеёиоуыэяю]', word)[1]) or (find[0]== 'в' and find[1]== re.findall(r'[^аеёиоуыэяю]', word)[1]) :
    return 'со'
  else:
    return 'с'

# приведение существительного в нужный падеж
def case(word_, c):
  word_ = morph.parse(word_)[0]
  word_in_case = word_.inflect({c}).word
  return word_in_case

# согласование существительного с числом
def with_number(n, word):
    parsed_word = morph.parse(word)[0]
    s = ' '.join([str(np.round(n,1)) ,parsed_word.make_agree_with_number(n).word])
    return s

In [0]:
# Формулировка прогноза для первого дня

city = 'Хабаровск' # город
sentences = list() # лист всех предложений

day_needed = name_of_day(df.columns[0])
day_needed = morph.parse(day_needed)[0]
day_needed = day_needed.inflect({'accs'}).word

mean_temp = np.mean([df.iloc[:,0][0],df.iloc[:,0][1]])
temperature = with_number(mean_temp,'градус')

city = case(city,'loct')
mpsec = with_number(df.iloc[:,0][2],'метр') + ' в секунду'
mms = with_number(df.iloc[:,0][3],'миллиметр')

s = ' '.join([preposition_v(day_needed).title(), day_needed, date(df.columns[0]), preposition_v(city), city.title(), 'будет', temperature]) + '. '
sentences.append(s)
s = ' '.join(['Скорость ветра будет достигать', mpsec]) + '. '
sentences.append(s)
s = ' '.join(['Уровень осадков будет равен', mms]) + '.'
sentences.append(s)

In [0]:
# Формулировка прогноза для всех остальных дней

def temp_diff(temp1,temp2):
  diff = np.abs(temp1-temp2)
  degree = with_number(diff, 'градус')
  if temp1 > temp2 : 
    return 'потеплеет на ' + degree 
  elif temp1 < temp2 : 
    return 'похолодает на ' + degree
  else : 
    return 'температура не изменится'
  
def precip_diff(mms1,mms2):
  diff = np.abs(mms1-mms2)
  count = with_number(diff, 'единица')
  if mms1 > mms2 : 
    return 'повысится на ' + count
  elif mms1 < mms2 : 
    return 'понизится на ' + count
  else : 
    return 'не изменится'

def sow_diff(sow1,sow2):
  diff = np.abs(sow1-sow2)
  count = with_number(diff,"единица")
  if diff>0 : 
    return 'изменится на ' + count
  else : 
    return 'не изменится'
  
# Метод

def forecast(i_col):
  day_one = name_of_day(df.columns[i_col])
  day_two = name_of_day(df.columns[i_col-1])
  date_one = date(df.columns[i_col])

  temp1 = np.mean([df.iloc[:,i_col][0],df.iloc[:,i_col][1]])
  temp2 = np.mean([df.iloc[:,i_col-1][0],df.iloc[:,i_col-1][1]])
  
  sow1 = df.iloc[:,i_col][2]
  sow2 = df.iloc[:,i_col-1][2]
  
  mms1 = df.iloc[:,i_col][3]
  mms2 = df.iloc[:,i_col-1][3]
  
  temp_s = list()
  
  s = ' '.join([preposition_v(day_one).title(), case(day_one,'accs'), date_one, preposition_v(city), city.title(), temp_diff(temp1,temp2), 'по сравнению', preposition_s(day_two), case(day_two, 'ablt')]) + '.'
  temp_s.append(s)
  
  if np.abs(temp1-temp2) != 0: 
    s = ' '.join(['Температура будет равна', with_number(temp1,'градус')]) + '.'
    temp_s.append(s)
    
  s = ' '.join(['Скорость ветра', sow_diff(sow1,sow2)]) +'.'
  temp_s.append(s)
  
  if np.abs(sow1-sow2) != 0: 
    s = ' '.join(['Ветер будет дуть со скоростью', with_number(sow1,'метр'), 'в секунду']) + '.'
    temp_s.append(s)
  
  s = ' '.join(['Уровень осадков', precip_diff(mms1,mms2)]) + '.'
  temp_s.append(s)
  
  if np.abs(mms1-mms2) != 0: 
    s = ' '.join(['Осадков выпадет', with_number(mms1,'миллиметр') ]) + '.'
    temp_s.append(s)
  
  return ' '.join(temp_s)

In [0]:
sentences.append('\n')

# Получение прогноза для всех дней
for i in range(1,len(df.columns)):
  sentences.append(forecast(i))
  sentences.append('\n')
  
# Последний день итог
temp1 = np.mean([df.iloc[:,9][0],df.iloc[:,9][1]])
temp2 = np.mean([df.iloc[:,0][0],df.iloc[:,0][1]])
mms1 = df.iloc[:,9][3]
mms2 = df.iloc[:,0][3]
s = ' '.join(['За', with_number(len(df.columns),'день'), preposition_v(city), city.title(), temp_diff(temp1,temp2)]) + '. '
sentences.append(s)
s = ' '.join(['Уровень осадков', precip_diff(mms1,mms2)]) + '.'
sentences.append(s)

### Полученный результат

In [0]:
# Итоговый прогноз
print(''.join(sentences))


В воскресенье 18 февраля в Хабаровске будет -17.5 градуса. Скорость ветра будет достигать 10.0 метров в секунду. Уровень осадков будет равен 0.0 миллиметров.
В понедельник 19 февраля в Хабаровске потеплеет на 4.0 градуса по сравнению с воскресеньем. Температура будет равна -13.5 градусов. Скорость ветра изменится на 3.0 единицы. Ветер будет дуть со скоростью 13.0 метров в секунду. Уровень осадков повысится на 0.1 единиц. Осадков выпадет 0.1 миллиметров.
Во вторник 20 февраля в Хабаровске похолодает на 2.0 градуса по сравнению с понедельником. Температура будет равна -15.5 градусов. Скорость ветра изменится на 1.0 единица. Ветер будет дуть со скоростью 12.0 метров в секунду. Уровень осадков понизится на 0.1 единиц. Осадков выпадет 0.0 миллиметров.
В среду 21 февраля в Хабаровске похолодает на 1.0 градус по сравнению со вторником. Температура будет равна -16.5 градуса. Скорость ветра изменится на 1.0 единица. Ветер будет дуть со скоростью 11.0 метров в секунду. Уровень осадков не изменит

### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

## Ответы на вопросы
### Вопрос №1
Генерировать текст по шаблонам может быть полезным в:
- типовых юридических документах, например различные иски и т.д., формируемые по типовым, стандартным шаблонам.
- почтовой рассылке с актуальными предложениями, в которой может быть персонализация - нужно только менять имена и прочую информацию.
- рекламных статьях, где на одну тематику сгенерировано множество различных формулировок.
- чат-ботах, которые могут из уже заложенных фраз и шаблонов формируют новые.

Генерировать текст об изменении числовых показателей по шаблонам может использоваться в:
- различных финансовых отчетах об изменениях ключевых показателей за год (например, размер чистой прибыли, коммерческого расхода)
- приложениях, отслеживающих какую-либо статистику (беговые приложения и т.п.)
***
### Вопрос №2
Первое, что можно сделать - это не высчитывать среднюю температуру между максимальной и минимальной, а использовать оба значения, таким образом сделав наш прогноз еще и более точным.

Разнообразить шаблоны можно с помощью использования синонимов, заменой часто употребляемых слов на них. Также можно объединять несколько шаблонов в одно предложение. Знание синтаксической структуры предоставит нам возможность менять словосочетания местами без потери смысла, тем самым создавать новые, уникальные предложения. Также, мы всегда можем создать новые шаблоны, используя другую синтаксическую структуру, но сохраняя смысл. Все предложенные способы можно использовать для генерации случайной последовательности шаблонов, которые будут образовывать взаимосвязанный текст, состоящий из разнообразных структур.

### Вывод
В ходе проделанной работы мы улучшили свои навыки владения библиотеками beautifulsoup4 и pymorphy2.
Мы реализовали алгоритм построения семантически верных предложений на основе скачанных данных. Данный алгоритм может быть применен для любого города, доступного на сайте gismeteo.ru - достаточно лишь ввести нужную ссылку.

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 17.02.2018.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 1: Анна Лапидус (anyalapidus@list.ru).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

### Как сдать домашнее задание в AnyTask
* Зарегистрируйтесь в системе AnyTask по ссылке http://anytask.org/accounts/register . Регистрация обязательна для всех!
* Подтвердите регистрацию по e-mail.
* Зайдите в свой профиль, нажмите “Активация инвайтов на курсы” и введите инвайт:

ИАД-1 : VNYQlGd

ИАД-2 : wDEa8ge

ИАД-3 : SviK5dd

ИАД-4 : dzdD8YG


 У вас появится курс “ВШЭ > Прикладные задачи анализа данных” в разделе “Посещает курсы”.
* Перейдите по ссылке “ВШЭ > Прикладные задачи анализа данных (2018)” и нажмите кнопку “Сдать”. 
* У вас откроется условие задачи и будут доступны различные поля, в частности, НИЖЕ условия задачи будет поле ввода, в которое вы сможете вписать какой-то комментарий, и сможете прикрепить файл. Сделайте это.
* Домашнее задание лучше всего сдавать в форматах IPYNB.
* Оценку вы получите также в системе AnyTask. За своей успеваемостью можете следить в разделе “Ведомость”, а также можете прокомментировать что-то в каждом вашем домашнем задании, зайдя на ее страничку (ячейки в табличке на страничке “Ведомость” кликабельны и ведут на ваш submission домашки).

(**ВАЖНО**) Если домашнее задание вы делали в группе, то в AnyTask домашнее сдает *один* участник группы, но заргестрироваться в AnyTask обязательно всем – так мы сможем проставить вам оценки в ведомость в AnyTask.

Ссылка на курс в AnyTask: http://anytask.org/course/281